In [1]:
import copy

import pandas as pd

from constraint_relax import search_mfs
from experiment_constraint_data_processor import process_experiment_constraint_data
from item_extrator import data_path
from similarity_calculator import compute_similarity
from utils import get_df_len

In [2]:
problem_list = process_experiment_constraint_data('./experiment_constraint_data.csv')

In [3]:
sol_list = []
result = pd.DataFrame()
threshold_list = ['', 0, 0.5, 1]
for problem in problem_list:
    prob_sol_list = dict()
    for threshold in threshold_list:
        p = copy.deepcopy(problem)
        if threshold != '':
            mfs = search_mfs(p.constraint_list, copy.copy(p.constraint_list), data_path)
            p.relax(mfs, threshold)
        prob_items = p.solve()
        prob_sol_list.update({
            f'constraint_{threshold}': p.constraint_list,
            f'no_of_items_{threshold}': get_df_len(prob_items)
        })
        if not prob_items is None:
            prob_sol_list.update({
                f'similarity_{threshold}': ((compute_similarity(p.constraint_list, prob_items))['similarity']).mean()
            })
        else:
            prob_sol_list.update({
                f'similarity_{threshold}': 0
            })
    result = result.append(prob_sol_list, ignore_index=True)

C:\Users\bttec\AppData\Local\Temp\ipykernel_2780\1560004041.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(prob_sol_list, ignore_index=True)
C:\Users\bttec\AppData\Local\Temp\ipykernel_2780\1560004041.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(prob_sol_list, ignore_index=True)
C:\Users\bttec\AppData\Local\Temp\ipykernel_2780\1560004041.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(prob_sol_list, ignore_index=True)
C:\Users\bttec\AppData\Local\Temp\ipykernel_2780\1560004041.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.appen

In [4]:
result

,constraint_,no_of_items_,similarity_,constraint_0,no_of_items_0,similarity_0,constraint_0.5,no_of_items_0.5,similarity_0.5,constraint_1,no_of_items_1,similarity_1
0,"{storage: 512.0: 7: Nature.MORE, os: 10: 4: Na...",14,0.765529,"{camera: : 1: Nature.EQUAL, price: 2699.0: 10:...",15,0.720075,"{storage: 512.0: 7: Nature.MORE, price: 2699.0...",15,0.720075,"{price: 2699.0: 10: Nature.LESS, os: 10: 4: Na...",15,0.720075
1,"{weight: 2.0: 9: Nature.LESS, camera: True: 10...",0,0.000000,"{ram: 8.0: 8: Nature.MORE, price: 3271.04: 6: ...",1,0.581119,"{ram: 8.0: 8: Nature.MORE, os: os: 8: Nature.E...",1,0.581119,"{os: os: 8: Nature.EQUAL, brand: : 7: Nature.E...",1,0.581119
2,"{price: 2664.04: 5: Nature.LESS, ram: 16.0: 9:...",0,0.000000,"{ram: 16.0: 9: Nature.MORE, storage: 128.0: 3:...",0,0.000000,"{price: 3214.04: 5: Nature.LESS, os: 11: 4: Na...",0,0.000000,"{price: : 5: Nature.LESS, storage: 128.0: 3: N...",4,0.487346
3,"{price: 4997.0: 3: Nature.LESS, brand: huawei|...",0,0.000000,{brand: huawei|dell|microsoft: 2: Nature.EQUAL...,0,0.000000,"{price: : 3: Nature.LESS, storage: 1024.0: 7: ...",0,0.000000,"{camera: True: 9: Nature.EQUAL, weight: : 6: N...",2,0.666667
4,"{weight: 2.3: 1: Nature.LESS, os: 11: 4: Natur...",5,0.629809,"{os: 11: 4: Nature.EQUAL, ram: 16.0: 7: Nature...",6,0.639378,"{os: 11: 4: Nature.EQUAL, cpu: ryzen 5|i7: 4: ...",6,0.624030,"{os: 11: 4: Nature.EQUAL, cpu: ryzen 5|i7: 4: ...",6,0.624030
...,...,...,...,...,...,...,...,...,...,...,...,...
495,"{storage: 1024.0: 1: Nature.MORE, ram: 4.0: 6:...",0,0.000000,"{storage: 512: 1: Nature.MORE, ram: 4.0: 6: Na...",0,0.000000,"{ram: 4.0: 6: Nature.MORE, os: os: 9: Nature.E...",3,0.607755,"{os: os: 9: Nature.EQUAL, brand: msi|acer|hp|h...",3,0.607755
496,"{ram: 4.0: 2: Nature.MORE, storage: 1024.0: 4:...",3,0.687003,"{weight: 1.47: 7: Nature.LESS, storage: 1024.0...",3,0.676014,"{ram: : 2: Nature.MORE, storage: 1024.0: 4: Na...",3,0.687003,"{storage: 1024.0: 4: Nature.MORE, weight: 1.47...",3,0.687003
497,"{storage: 256.0: 6: Nature.MORE, weight: 1.44:...",9,0.507405,"{camera: True: 1: Nature.EQUAL, weight: 1.5414...",10,0.502405,"{camera: : 1: Nature.EQUAL, weight: 1.44: 1: N...",10,0.407405,"{weight: : 1: Nature.LESS, os: 11: 2: Nature.E...",30,0.435484
498,"{storage: 1024.0: 5: Nature.MORE, brand: lenov...",0,0.000000,"{os: 10: 10: Nature.EQUAL, storage: 1024.0: 5:...",0,0.000000,"{os: 10: 10: Nature.EQUAL, storage: 1024.0: 5:...",1,0.920000,"{storage: 1024.0: 5: Nature.MORE, brand: lenov...",1,0.920000


In [5]:
pd.value_counts(result.iloc[:, 1] == 0)

True     285
False    215
Name: no_of_items_, dtype: int64

In [6]:
valid = ((result.iloc[:, 1] <= result.iloc[:, 4]) & (result.iloc[:, 4] <= result.iloc[:, 7]) & (result.iloc[:, 7] <= result.iloc[:, 10]))
valid_count = pd.value_counts(valid)
print(valid_count)

True     471
False     29
dtype: int64


In [7]:
valid = ((result.iloc[:, 1] < result.iloc[:, 4]) & (result.iloc[:, 4] < result.iloc[:, 7]) & (result.iloc[:, 7] < result.iloc[:, 10]))
valid_count = pd.value_counts(valid)
print(valid_count)

False    498
True       2
dtype: int64


In [8]:
result[valid]

,constraint_,no_of_items_,similarity_,constraint_0,no_of_items_0,similarity_0,constraint_0.5,no_of_items_0.5,similarity_0.5,constraint_1,no_of_items_1,similarity_1
229,"{weight: 2.47: 4: Nature.LESS, price: 4706.33:...",12,0.661046,"{weight: 2.47: 4: Nature.LESS, os: 10: 6: Natu...",14,0.658962,"{price: : 1: Nature.LESS, brand: lenovo|huawei...",16,0.633422,"{weight: 2.47: 4: Nature.LESS, os: 10: 6: Natu...",47,0.619379
482,"{weight: 1.52: 2: Nature.LESS, ram: 8.0: 4: Na...",0,0.000000,"{ram: 8.0: 4: Nature.MORE, storage: 512: 5: Na...",6,0.674799,"{storage: 512: 5: Nature.MORE, price: 5056.09:...",8,0.595528,"{ram: 8.0: 4: Nature.MORE, storage: : 5: Natur...",9,0.527914
